In [12]:
!pip install wandb -qqq
!pip install -q simpletransformers
!pip install requests
!pip install sacremoses
import wandb
import pandas as pd
import sklearn
wandb.login()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


wandb: Currently logged in as: s20783. Use `wandb login --relogin` to force relogin


True

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/')

Mounted at /content/drive


In [13]:
df = pd.read_csv("daneINL.csv", encoding="ISO-8859-1")

def preprocess_file():
  a = df[df.iloc[:, 0] != df.iloc[:, 1]]
  tekst = a.iloc[:, 0].tolist() + a.iloc[:, 1].tolist()
  label = [0] * len(a) + [1] * len(a)
  new_df = pd.DataFrame({'Tekst': tekst, 'Label': label})
  return new_df

df = preprocess_file()
print(df)

train_ratio = 0.75
dev_ratio = 0.15
test_ratio = 0.1

train_df, temp_df = train_test_split(df, test_size=(1 - train_ratio), random_state=32)
eval_df, test_df = train_test_split(temp_df, test_size=test_ratio/(test_ratio + dev_ratio), random_state=32)

model_type = "herbert"
model_name = "allegro/herbert-base-cased"
# model_type = "distilbert"
# model_name = "distilbert-base-uncased"

train_args = {
    'evaluate_during_training': True,
    'num_train_epochs': 4,
    'save_eval_checkpoints': False,
    'train_batch_size': 8,
    'eval_batch_size': 8,
    'overwrite_output_dir': True,
    'wandb_project': "INL_projekt_s20783",
}

model = ClassificationModel(
    model_type,
    model_name,
    use_cuda=False,
    cuda_device=0,
    num_labels=2,
    args=train_args
)

model.train_model(train_df, eval_df=eval_df)
result, model_outputs, wrong_predictions = model.eval_model(test_df)
print(result)


dev_predictions, dev_raw_outputs = model.predict(eval_df.iloc[:,0].tolist())
dev_f1 = f1_score(eval_df.iloc[:,1].tolist(), dev_predictions)
dev_accuracy = accuracy_score(eval_df.iloc[:,1].tolist(), dev_predictions)

print("F1-Score:", dev_f1)
print("Accuracy:", dev_accuracy)


test_predictions, test_raw_outputs = model.predict(test_df.iloc[:,0].tolist())
test_f1 = f1_score(test_df.iloc[:,1].tolist(), test_predictions)
test_accuracy = accuracy_score(test_df.iloc[:,1].tolist(), test_predictions)

print("Test F1-Score:", test_f1)
print("Test Accuracy:", test_accuracy)

               Tekst  Label
0         Nauczyciel      0
1             Lekarz      0
2            Student      0
3         Sprzedawca      0
4           In¿ynier      0
..               ...    ...
223        Hokeistka      1
224        Cielanka      1
225         Krawcowa      1
226  Sêdzi pi³karska      1
227            DJ-ka      1

[228 rows x 2 columns]


Some weights of the model checkpoint at allegro/herbert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.sso.sso_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.sso.sso_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification 

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/171 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/5 [00:00<?, ?it/s]

F1-Score: 1.0
Accuracy: 1.0


  0%|          | 0/23 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Test F1-Score: 0.9565217391304348
Test Accuracy: 0.9565217391304348


In [39]:
def sprawdz_forme(text):
  predictions, raw_outputs = model.predict([text])
  return predictions

user_input = input("Wpisz słowo: ")
out = sprawdz_forme(user_input)
if(out == [0]):
  print("Forma męska")
else:
  print("Forma żeńska")

Wpisz słowo: nauczycielka


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Forma żeńska
